In [ ]:
%matplotlib inline


# Source localization with single dipole fit


The aim of this lecture is to show you how to do sequential and
fixed dipole fitting with MNE-Python.

In [ ]:
from os import path as op
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.forward import make_forward_dipole
from mne.evoked import combine_evoked
from mne.simulation import simulate_evoked

# data_path = sample.data_path()
data_path = '/Users/alex/mne_data/MNE-sample-data'

subjects_dir = op.join(data_path, 'subjects')
fname_ave = op.join(data_path, 'MEG', 'sample', 'sample_audvis-ave.fif')
fname_cov = op.join(data_path, 'MEG', 'sample', 'sample_audvis-cov.fif')
fname_bem = op.join(subjects_dir, 'sample', 'bem', 'sample-5120-bem-sol.fif')
fname_trans = op.join(data_path, 'MEG', 'sample',
                      'sample_audvis_raw-trans.fif')
fname_surf_lh = op.join(subjects_dir, 'sample', 'surf', 'lh.white')

Let's localize the N100m (using MEG only):



In [ ]:
evoked = mne.read_evokeds(fname_ave, condition='Left Auditory', baseline=(None, 0))
evoked.pick_types(meg=True, eeg=False)

In [ ]:
# Fit a dipole using a sequential (time-varying position and orientation) fit
evoked_crop = evoked.copy().crop(0.075, 0.09)
dip, residual = mne.fit_dipole(evoked_crop, fname_cov, fname_bem,
                               fname_trans)
print(dip)

In [ ]:
# Look at our result
print(dip.gof)

In [ ]:
selection = mne.read_selection('Right', info=evoked.info)
# Fit a dipole using a sequential (time-varying position and orientation) fit
evoked_crop_right = evoked.copy().crop(0.075, 0.09)
evoked_crop_right.pick_channels(selection)
dip, residual = mne.fit_dipole(evoked_crop_right, fname_cov, fname_bem,
                               fname_trans)
print(dip)

In [ ]:
dip.gof

Calculate and visualise magnetic field predicted by dipole with maximum GOF
and compare to the measured data, highlighting the ipsilateral (right) source



In [ ]:
# find time point with highest GOF
best_idx = np.argmax(dip.gof)
best_time = dip.times[best_idx]
best_gof = dip.gof[best_idx]
best_pos = dip.pos[best_idx]
print('t=%0.3f, gof=%0.1f%%, pos=%s' % (best_time, best_gof, best_pos))

Estimate the time course of our best-GOF dipole by using a fixed position
and orientation (the one that maximized GOF) over the entire interval

In [ ]:
best_ori = dip.ori[best_idx]
dip_fixed = mne.fit_dipole(evoked, fname_cov, fname_bem, fname_trans,
                           pos=best_pos, ori=best_ori)[0]
dip_fixed.plot();

In [ ]:
trans = data_path + '/MEG/sample/sample_audvis_raw-trans.fif'
subjects_dir = data_path + '/subjects'
dip.plot_locations(trans=trans, subject='sample',
                   subjects_dir=subjects_dir, mode='orthoview');

## Exercises
- Try to improve the dipole fit by using a subselection of channels:
    - Subselect channels from ``evoked`` and ``evoked_crop``
      (think about ``mne.read_selection(...)``, ``evoked.pick_channels(...)``)
    - Do a time-varying dipole fit of ``evoked_crop``
    - Do a fixed-position, fixed-orientation fit using ``evoked`` and visualize the result
- What is the improvement in max GOF from using a subselection of channels?
- How far (in mm) is the new best dipole from the original best dipole?